<a href="https://colab.research.google.com/github/jbassie/WEB-SCRAPING/blob/main/REAL_ESTATE/Airbnb_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
#Selenium Import
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubunt

In [105]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [106]:
import time
from random import randint
from bs4 import BeautifulSoup
import requests #to connect to urlbb
from lxml import etree
import pandas as pd

In [107]:
data = pd.read_csv('/content/sample_data/Airbnb_Data.csv')
data.head()

,address,location/lat,location/lng,name,numberOfGuests,roomType,stars,url
0,"Santa Cruz, Aruba",12.50148,-69.98387,Casa Noa -modern house (2021) with private pool,4,Entire bungalow,5.0,https://www.airbnb.com/rooms/51623850
1,"Oranjestad, Aruba",12.50484,-70.00170,"“comfortable apartment with great location""",2,Entire rental unit,5.0,https://www.airbnb.com/rooms/22586430
2,"Noord, Aruba",12.57936,-70.04427,ABSC3 ☆ Marriott's Aruba Surf Club ☆ 3BR Villa ☆,12,Entire condo,NaN,https://www.airbnb.com/rooms/43132908
3,"Oranjestad, Aruba",12.52670,-70.03065,"Kamerlingh Villa - The Villa, Casa Curaçao & A...",16,Entire villa,NaN,https://www.airbnb.com/rooms/45186681
4,"Noord, Aruba",12.57884,-70.04448,3 BEDROOM SUITE MARRIOTT'S ARUBA SURF CLUB!,12,Private room in resort,NaN,https://www.airbnb.com/rooms/52433779


In [112]:
link = data['url']
link = link.to_list()


['https://www.airbnb.com/rooms/51623850',
 'https://www.airbnb.com/rooms/22586430',
 'https://www.airbnb.com/rooms/43132908',
 'https://www.airbnb.com/rooms/45186681',
 'https://www.airbnb.com/rooms/52433779',
 'https://www.airbnb.com/rooms/719765512737851800',
 'https://www.airbnb.com/rooms/31695438',
 'https://www.airbnb.com/rooms/45185807',
 'https://www.airbnb.com/rooms/47812607',
 'https://www.airbnb.com/rooms/11289864',
 'https://www.airbnb.com/rooms/46108736',
 'https://www.airbnb.com/rooms/8533916',
 'https://www.airbnb.com/rooms/22810770',
 'https://www.airbnb.com/rooms/28338466',
 'https://www.airbnb.com/rooms/26471884',
 'https://www.airbnb.com/rooms/50867789',
 'https://www.airbnb.com/rooms/52101217',
 'https://www.airbnb.com/rooms/22429458',
 'https://www.airbnb.com/rooms/47705953',
 'https://www.airbnb.com/rooms/37037441',
 'https://www.airbnb.com/rooms/53143779',
 'https://www.airbnb.com/rooms/49482614',
 'https://www.airbnb.com/rooms/23921053',
 'https://www.airbnb.com/

In [130]:
def get_details(link):
    property_list = []
    word = 'pool'
    for url in link:
        driver.get(url)
        #print(driver.title)
        time.sleep(2)
        try:
            name  = driver.find_elements(By.CLASS_NAME, '_fecoyn4')[0].text
            Reviews  = driver.find_elements(By.CLASS_NAME, '_11eqlma4')[0].text
            location = driver.find_elements(By.CLASS_NAME, '_9xiloll')[0].text
            timeout = 10
            expectation = EC.presence_of_element_located((By.CSS_SELECTOR, '._tyxjp1'))
            price = WebDriverWait(driver, timeout).until(expectation).get_attribute('innerHTML').split('&')[0]
            bedrooms = driver.find_elements(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[2]/span[2]')[0].text
            host = driver.find_elements(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[1]/div[2]/h2')[0].text
            beds = driver.find_elements(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[3]/span[2]')[0].text
            bathrooms =  driver.find_elements(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[4]/span[2]')[0].text
            pool =  driver.find_elements(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[4]/div/div[2]/div[1]/div/span/span')[0].text
        except IndexError:
            pass
        
        prop = {
            'name': name,
            'location': location,
            'price': price,
            'bedrooms': str(bedrooms) + "(" + str(beds )+ ")",
            'host': host,
            'bathrooms' : bathrooms,
            'Pool': [ 'Yes' if word in pool else 'No'],
            'link': url
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    print(df.head(20))
    df.to_csv('Airbnb_dataset.csv', header=True, index=False)


In [131]:
get_details(link[:30])

Saving: Casa Noa -modern house (2021) with private pool
Saving: Casa Noa -modern house (2021) with private pool
Saving: ABSC3 ☆ Marriott's Aruba Surf Club ☆ 3BR Villa ☆
Saving: Kamerlingh Villa - The Villa, Casa Curaçao & Apt 9
Saving: 3 BEDROOM SUITE MARRIOTT'S ARUBA SURF CLUB!
Saving: 3/3 Bedroom Suite Divi Aruba Phoenix Beach Club!
Saving: Marriott's Aruba Surf and Ocean Clubs Resorts 2
Saving: Kamerlingh Villa
Saving: ABSC3 ☆ Marriott's Aruba Surf Club ☆ 3BR Villa ☆
Saving: Directly from owner: Villa Aruba Mansion 25 pers.
Saving: Best PalmBeach Villa26ppl A chef service Included
Saving: Magnificent mansion with million dollar view of Aruba
Saving: Oceafront 5 BDR Penthouse like no other in Aruba!
Saving: 5-Star Casa del Viento Aruba Beach Villa
Saving: 5-Star Casa del Viento Aruba Beach Villa
Saving: NEW ! Exclusive & Luxurious Beach Front Mansion
Saving: A place for everyone, designer's corner-Penthouse with BBQ
Saving: New Aruba Beachfront Estate:gym, beach and pool
Saving: Styl